In [6]:
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 54.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 95.6 MB/s eta 0:00:00:00:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
# В Jupyter Notebook/JupyterLab:
%pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiobotocore                      2.1.2
aiohttp                          3.8.5
aiohttp-retry                    2.8.3
aioitertools                     0.11.0
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.2.1
altair                           4.2.2
amqp                             5.1.1
antlr4-python3-runtime           4.9.3
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      21.3.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.0
arviz                            0.15.1
astropy                          5.2.2
astunparse                       1.6.3
async-timeout                    4.0.2
asyncssh                         2.13.2
atpublic                         4.0
attrs                            23.1.0
au

In [18]:



try:
    from transformers import BertTokenizer, BertModel
    import torch
    print("Библиотеки успешно загружены!")
except ImportError:
    print("Установка необходимых библиотек...")
    
    # Пробуем разные способы установки
    try:
        get_ipython().run_line_magic('pip', 'install transformers torch')
    except:
        try:
            import sys
            import subprocess
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers', 'torch'])
        except:
            print("""
            Не удалось установить автоматически. Пожалуйста:
            1. Откройте терминал/командную строку
            2. Выполните: pip install transformers torch
            3. Перезапустите Jupyter Notebook
            """)
            raise

# Теперь основной код
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Остальной ваш код...

# Загрузка предобученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)
model.eval()  # Переключаем в режим оценки

# Функция для получения внимания для предложения
def get_attention_for_text(text):
    # Токенизация текста
    encoded_input = tokenizer(text, return_tensors='pt')
    
    # Прямой проход через модель
    with torch.no_grad():
        outputs = model(**encoded_input)
    
    # Получаем матрицы внимания
    # attention имеет размерность [batch_size, num_heads, seq_length, seq_length]
    attention = outputs.attentions
    
    # Получаем токены
    tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
    
    return attention, tokens

# Функция для визуализации внимания
def visualize_attention(attention, tokens, layer=0, head=0):
    # Получаем матрицу внимания для конкретного слоя и головы
    attention_matrix = attention[layer][0, head].numpy()
    
    # Создаем тепловую карту
    plt.figure(figsize=(10, 8))
    sns.heatmap(attention_matrix, 
                xticklabels=tokens, 
                yticklabels=tokens, 
                cmap='viridis', 
                annot=False)
    
    plt.title(f'Внимание для слоя {layer+1}, головы {head+1}', fontsize=14)
    plt.xlabel('Токены', fontsize=12)
    plt.ylabel('Токены', fontsize=12)
    plt.tight_layout()
    plt.show()

# Функция для визуализации внимания с графом
def visualize_attention_graph(attention, tokens, layer=0, head=0, threshold=0.1):
    import networkx as nx
    
    # Получаем матрицу внимания для конкретного слоя и головы
    attention_matrix = attention[layer][0, head].numpy()
    
    # Создаем направленный граф
    G = nx.DiGraph()
    
    # Добавляем узлы
    for i, token in enumerate(tokens):
        G.add_node(i, label=token)
    
    # Добавляем рёбра с весами выше порога
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            weight = attention_matrix[i, j]
            if weight > threshold:
                G.add_edge(i, j, weight=weight)
    
    # Рисуем граф
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(G, k=0.9)
    
    # Рисуем узлы
    nx.draw_networkx_nodes(G, pos, node_size=700, node_color='lightblue')
    
    # Рисуем метки узлов
    nx.draw_networkx_labels(G, pos, labels={i: G.nodes[i]['label'] for i in G.nodes})
    
    # Рисуем рёбра с разной толщиной в зависимости от веса
    edge_weights = [G[u][v]['weight'] * 10 for u, v in G.edges()]
    nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.7, edge_color='gray', 
                          connectionstyle='arc3,rad=0.1', arrowsize=15)
    
    plt.title(f'Граф внимания для слоя {layer+1}, головы {head+1} (порог = {threshold})', fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Создаем интерактивную визуализацию внимания для всех голов и слоев
def interactive_attention_visualization(text):
    attention, tokens = get_attention_for_text(text)
    num_layers = len(attention)
    num_heads = attention[0].size(1)
    
    # Создаем интерактивные элементы управления (для Jupyter Notebook)
    from ipywidgets import interact, widgets
    
    @interact(
        layer=widgets.IntSlider(min=0, max=num_layers-1, step=1, value=0, description='Слой:'),
        head=widgets.IntSlider(min=0, max=num_heads-1, step=1, value=0, description='Голова:'),
        threshold=widgets.FloatSlider(min=0.0, max=0.5, step=0.01, value=0.1, description='Порог:')
    )
    def plot_attention(layer, head, threshold):
        plt.figure(figsize=(15, 10))
        
        # Тепловая карта внимания
        plt.subplot(1, 2, 1)
        att_matrix = attention[layer][0, head].numpy()
        sns.heatmap(att_matrix, xticklabels=tokens, yticklabels=tokens, cmap='viridis')
        plt.title(f'Тепловая карта внимания (слой {layer+1}, голова {head+1})')
        
        # Граф внимания
        plt.subplot(1, 2, 2)
        import networkx as nx
        
        # Создаем граф
        G = nx.DiGraph()
        for i, token in enumerate(tokens):
            G.add_node(i, label=token)
        
        # Добавляем рёбра с весами выше порога
        for i in range(len(tokens)):
            for j in range(len(tokens)):
                weight = att_matrix[i, j]
                if weight > threshold:
                    G.add_edge(i, j, weight=weight)
        
        # Рисуем граф
        pos = nx.spring_layout(G, k=0.9)
        nx.draw_networkx_nodes(G, pos, node_size=700, node_color='lightblue')
        nx.draw_networkx_labels(G, pos, labels={i: G.nodes[i]['label'] for i in G.nodes})
        edge_weights = [G[u][v]['weight'] * 10 for u, v in G.edges()]
        nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.7, edge_color='gray',
                              connectionstyle='arc3,rad=0.1', arrowsize=15)
        
        plt.title(f'Граф внимания (порог = {threshold})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return plot_attention

# Пример использования
text = "The cat sat on the mat because it was tired."
attention, tokens = get_attention_for_text(text)

# Визуализация внимания для конкретного слоя и головы
visualize_attention(attention, tokens, layer=5, head=7)

# Визуализация графа внимания
visualize_attention_graph(attention, tokens, layer=5, head=7, threshold=0.1)

# Создание интерактивной визуализации (для Jupyter Notebook)
# interactive_viz = interactive_attention_visualization(text)
# display(interactive_viz)

# Визуализация агрегированного внимания по всем головам для конкретного слоя
def visualize_aggregated_attention(attention, tokens, layer=0):
    num_heads = attention[layer].size(1)
    # Усредняем внимание по всем головам
    avg_attention = torch.mean(attention[layer][0], dim=0).numpy()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(avg_attention, 
                xticklabels=tokens, 
                yticklabels=tokens, 
                cmap='viridis', 
                annot=False)
    
    plt.title(f'Агрегированное внимание для слоя {layer+1} (среднее по {num_heads} головам)', fontsize=14)
    plt.xlabel('Токены', fontsize=12)
    plt.ylabel('Токены', fontsize=12)
    plt.tight_layout()
    plt.show()

# Визуализация агрегированного внимания для последнего слоя
visualize_aggregated_attention(attention, tokens, layer=-1)

# Визуализация внимания токена к себе по слоям
def visualize_self_attention_by_layer(attention, tokens):
    num_layers = len(attention)
    num_tokens = len(tokens)
    
    # Собираем самовнимание для каждого токена по слоям
    self_attention = np.zeros((num_layers, num_tokens))
    
    for layer in range(num_layers):
        # Усредняем по всем головам
        avg_attention = torch.mean(attention[layer][0], dim=0).numpy()
        
        self_attention[layer] = np.diag(avg_attention)
    
    # Визуализация
    plt.figure(figsize=(14, 8))
    sns.heatmap(self_attention, 
                xticklabels=tokens, 
                yticklabels=[f'Слой {i+1}' for i in range(num_layers)], 
                cmap='viridis', 
                annot=False)
    
    plt.title('Самовнимание токенов по слоям', fontsize=14)
    plt.xlabel('Токены', fontsize=12)
    plt.ylabel('Слои', fontsize=12)
    plt.tight_layout()
    plt.show()

# Визуализация самовнимания по слоям
visualize_self_attention_by_layer(attention, tokens) 

Установка необходимых библиотек...
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


ModuleNotFoundError: No module named 'transformers'

In [22]:
%pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
